# Megaton E2E Smoke Test (Manual)

**目的**
- Megaton を使った最小の end-to-end 動作確認（外部サービスにアクセスします）

**これは保証しません**
- 網羅的な正しさ・パフォーマンス・権限設定の完全性
- GA4 / Sheets / BQ のすべての組み合わせ

**安全メモ**
- Google Sheets への書き込みは **新しい一時シート** を作成して行います（既存シートは上書きしません）。

In [ ]:
# Cell 0: Configuration (single place to edit)
import os
from datetime import datetime, timedelta, timezone

RUN_GA4 = True
RUN_SHEETS = True
RUN_BQ = True
RUN_GSC = False  # optional
RUN_DRIVE = False  # optional
RUN_UI_SMOKE = False  # optional
RUN_OAUTH_SMOKE = False  # optional

OAUTH_JSON = None  # optional override for OAuth JSON (string or path)

# 1-day range by default (UTC)
DATE_TO = (datetime.now(timezone.utc) - timedelta(days=1)).strftime("%Y-%m-%d")
DATE_FROM = DATE_TO

# Optional env-based settings (loaded after .env in Cell 2 if empty)
GS_URL = os.environ.get("SMOKE_GS_URL", "")
BQ_PROJECT = os.environ.get("SMOKE_BQ_PROJECT", "")
BQ_DATASET = os.environ.get("SMOKE_BQ_DATASET", "")
GSC_SITE_URL = os.environ.get("SMOKE_GSC_SITE_URL", "")

TEMP_SHEET_NAME = f"_smoke_{datetime.now(timezone.utc).strftime('%Y%m%d_%H%M')}"
MEGATON_GIT_REF = os.environ.get("MEGATON_GIT_REF", "main")

SMOKE_RESULTS = {}
GATE_OK = True  # required checks must keep this True


In [ ]:
# Cell 1: Environment detection + install
import sys
import subprocess

IN_COLAB = "google.colab" in sys.modules
MEGATON_AVAILABLE = False


def _pip_install(args):
    cmd = [sys.executable, "-m", "pip"] + args
    print("Running:", " ".join(cmd))
    subprocess.check_call(cmd)


try:
    import megaton  # noqa: F401
    MEGATON_AVAILABLE = True
except Exception:
    if IN_COLAB:
        pkg = f"git+https://github.com/mak00s/megaton@{MEGATON_GIT_REF}"
        _pip_install(["install", "-U", pkg])
        try:
            import megaton  # noqa: F401
            MEGATON_AVAILABLE = True
        except Exception as exc:
            print("Failed to import megaton after install:", exc)
    else:
        print("megaton is not installed. Trying editable install: pip install -e .")
        try:
            _pip_install(["install", "-e", "."])
            import megaton  # noqa: F401
            MEGATON_AVAILABLE = True
        except Exception as exc:
            print("Install failed. Please run: pip install -e .")
            print("Error:", exc)

print("IN_COLAB:", IN_COLAB)
print("MEGATON_AVAILABLE:", MEGATON_AVAILABLE)
print("MEGATON_GIT_REF:", MEGATON_GIT_REF)

In [ ]:
# Cell 2: Load .env (simple)
import os

CREDS_JSON = None
READY = False

if not MEGATON_AVAILABLE:
    print("Skipped: megaton is not available.")
    SMOKE_RESULTS["env"] = "FAIL (megaton not available)"
    GATE_OK = False
else:
    load_dotenv = None
    try:
        from dotenv import load_dotenv  # type: ignore
    except Exception:
        if IN_COLAB:
            try:
                _pip_install(["install", "python-dotenv"])
                from dotenv import load_dotenv  # type: ignore
            except Exception as exc:
                print("Failed to install python-dotenv:", exc)
        else:
            print("python-dotenv is not installed; skipping .env load.")

    if load_dotenv:
        load_dotenv(".env")

    # Refresh config values if they were empty
    if not GS_URL:
        GS_URL = os.environ.get("SMOKE_GS_URL", "")
    if not BQ_PROJECT:
        BQ_PROJECT = os.environ.get("SMOKE_BQ_PROJECT", "")
    if not BQ_DATASET:
        BQ_DATASET = os.environ.get("SMOKE_BQ_DATASET", "")
    if not GSC_SITE_URL:
        GSC_SITE_URL = os.environ.get("SMOKE_GSC_SITE_URL", "")

    CREDS_JSON = os.environ.get("MEGATON_CREDS_JSON")

    if not CREDS_JSON:
        print("MEGATON_CREDS_JSON is not set.")
        print("Create .env and set MEGATON_CREDS_JSON to a single-line JSON string.")
        print("If you prefer multiline JSON, set CREDS_JSON manually in this cell.")
        READY = False
        SMOKE_RESULTS["env"] = "FAIL (MEGATON_CREDS_JSON missing)"
        GATE_OK = False
    else:
        READY = True
        SMOKE_RESULTS["env"] = "OK"

# Optional: set CREDS_JSON manually here (do not print it)
# CREDS_JSON = '{...}'

In [ ]:
# Cell 3: Initialize Megaton
from megaton.start import Megaton
import pandas as pd

mg = None

if not READY:
    print("Skipped: credentials are not configured.")
else:
    try:
        mg = Megaton(CREDS_JSON, headless=True)
        print("Megaton initialized.")
        SMOKE_RESULTS["init"] = "OK"
    except Exception as exc:
        print("Failed to initialize Megaton:", exc)
        err_msg = str(exc)
        if "NameResolutionError" in err_msg or "oauth2.googleapis.com" in err_msg:
            print("Network/DNS error while contacting OAuth endpoint.")
            print("Check connectivity or proxy settings, then retry.")
            print("If using OAuth, run once with headless=False to create a cache.")
        SMOKE_RESULTS["init"] = "FAIL (init error)"
        GATE_OK = False
        mg = None


def _select_first_ga4_property(mg_instance):
    ga4_client = mg_instance.ga.get("4") if mg_instance else None
    if not ga4_client:
        return False
    if not ga4_client.accounts:
        return False
    account_id = ga4_client.accounts[0]["id"]
    ga4_client.account.select(account_id)
    if not ga4_client.account.properties:
        return False
    property_id = ga4_client.account.properties[0]["id"]
    ga4_client.property.select(property_id)
    print("Selected GA4 property:", property_id)
    return True

In [ ]:
# Cell 4: GA4 smoke (required if RUN_GA4)
if not RUN_GA4:
    print("Skipped: RUN_GA4 is False")
    SMOKE_RESULTS["ga4"] = "SKIP"
elif not mg:
    print("Skipped: Megaton not initialized")
    SMOKE_RESULTS["ga4"] = "FAIL (no Megaton)"
    GATE_OK = False
else:
    if not mg.ga.get("4"):
        print("Skipped: GA4 client is not available")
        SMOKE_RESULTS["ga4"] = "FAIL (no GA4 client)"
        GATE_OK = False
    else:
        if not mg.ga["4"].property.id:
            if not _select_first_ga4_property(mg):
                print("Skipped: no GA4 properties available")
                SMOKE_RESULTS["ga4"] = "FAIL (no GA4 properties)"
                GATE_OK = False
        if mg.ga["4"].property.id:
            try:
                mg.report.set_dates(DATE_FROM, DATE_TO)
                mg.report.run(
                    d=["date", "eventName"],
                    m=["eventCount"],
                )
                df = mg.report.data
                print("GA4 shape:", df.shape)
                display(df.head())
                assert isinstance(df, pd.DataFrame)
                assert {"date", "eventName", "eventCount"}.issubset(df.columns)
                SMOKE_RESULTS["ga4"] = "OK"
            except Exception as exc:
                print("GA4 smoke failed:", exc)
                SMOKE_RESULTS["ga4"] = "FAIL (query/assert)"
                GATE_OK = False

In [ ]:
# Cell 5: Sheets smoke (optional if RUN_SHEETS)
if not RUN_SHEETS:
    print("Skipped: RUN_SHEETS is False")
    SMOKE_RESULTS["sheets"] = "SKIP"
elif not GATE_OK:
    print("Skipped: required checks failed")
    SMOKE_RESULTS["sheets"] = "SKIP (gate)"
elif not mg:
    print("Skipped: Megaton not initialized")
    SMOKE_RESULTS["sheets"] = "SKIP (no Megaton)"
elif not GS_URL:
    print("Skipped: GS_URL not set")
    SMOKE_RESULTS["sheets"] = "SKIP (no GS_URL)"
else:
    try:
        mg.open.sheet(GS_URL)
        existing = set(mg.gs.sheets or [])
        name = TEMP_SHEET_NAME
        counter = 1
        while name in existing:
            name = f"{TEMP_SHEET_NAME}_{counter}"
            counter += 1
        mg.gs.sheet.create(name)
        df = mg.report.data if mg and mg.report.data is not None else pd.DataFrame({"test": [1]})
        mg.save.to.sheet(name, df)
        print("Wrote to new sheet:", name)
        SMOKE_RESULTS["sheets"] = f"OK ({name})"
    except Exception as exc:
        print("Sheets smoke failed:", exc)
        SMOKE_RESULTS["sheets"] = "FAIL (write error)"

In [ ]:
# Cell 6: BigQuery smoke (optional if RUN_BQ)
if not RUN_BQ:
    print("Skipped: RUN_BQ is False")
    SMOKE_RESULTS["bq"] = "SKIP"
elif not GATE_OK:
    print("Skipped: required checks failed")
    SMOKE_RESULTS["bq"] = "SKIP (gate)"
elif not mg:
    print("Skipped: Megaton not initialized")
    SMOKE_RESULTS["bq"] = "SKIP (no Megaton)"
elif not BQ_PROJECT:
    print("Skipped: BQ_PROJECT not set")
    SMOKE_RESULTS["bq"] = "SKIP (no BQ_PROJECT)"
else:
    try:
        bq_client = mg.launch_bigquery(BQ_PROJECT)
        if not bq_client:
            print("Skipped: BigQuery client not initialized")
            SMOKE_RESULTS["bq"] = "SKIP (no client)"
        else:
            df = bq_client.run("SELECT 1 AS test", to_dataframe=True)
            print("BQ result:")
            display(df)
            if BQ_DATASET:
                bq_client.dataset.select(BQ_DATASET)
                date1 = DATE_FROM.replace("-", "")
                date2 = DATE_TO.replace("-", "")
                sql = bq_client.ga4.get_query_to_flatten_events(date1, date2, event_parameters=[], user_properties=[])
                print("Flatten SQL (preview):")
                print(sql[:400])
            SMOKE_RESULTS["bq"] = "OK"
    except Exception as exc:
        err_msg = str(exc)
        if "bigquery.jobs.create" in err_msg or "Access Denied" in err_msg or "permission" in err_msg.lower():
            print("Skipped: BigQuery job permission is missing.")
            SMOKE_RESULTS["bq"] = "SKIP (permission)"
        else:
            print("BigQuery smoke failed:", exc)
            SMOKE_RESULTS["bq"] = "FAIL (query error)"

In [ ]:
# Cell 7: Optional services (GSC / Drive)
if RUN_GSC:
    if not GATE_OK:
        print("Skipped: required checks failed")
        SMOKE_RESULTS["gsc"] = "SKIP (gate)"
    elif not mg:
        print("Skipped: Megaton not initialized")
        SMOKE_RESULTS["gsc"] = "SKIP (no Megaton)"
    elif not GSC_SITE_URL:
        print("Skipped: GSC_SITE_URL not set")
        SMOKE_RESULTS["gsc"] = "SKIP (no GSC_SITE_URL)"
    else:
        try:
            sc = mg.launch_sc(GSC_SITE_URL)
            if sc:
                print("Search Console client initialized.")
                SMOKE_RESULTS["gsc"] = "OK"
        except Exception as exc:
            print("Search Console smoke failed:", exc)
            SMOKE_RESULTS["gsc"] = "FAIL (init error)"
else:
    print("Skipped: RUN_GSC is False")
    SMOKE_RESULTS["gsc"] = "SKIP"

if RUN_DRIVE:
    try:
        from megaton import mount_google_drive

        mount_google_drive()
        SMOKE_RESULTS["drive"] = "OK"
    except Exception as exc:
        print("Drive mount failed:", exc)
        SMOKE_RESULTS["drive"] = "FAIL (mount error)"
else:
    print("Skipped: RUN_DRIVE is False")
    SMOKE_RESULTS["drive"] = "SKIP"

if RUN_UI_SMOKE and RUN_OAUTH_SMOKE:
    print("Skipped: RUN_UI_SMOKE disabled because RUN_OAUTH_SMOKE is enabled")
    SMOKE_RESULTS["ui"] = "SKIP (covered by oauth)"
elif RUN_UI_SMOKE:
    if not GATE_OK:
        print("Skipped: required checks failed")
        SMOKE_RESULTS["ui"] = "SKIP (gate)"
    elif not mg:
        print("Skipped: Megaton not initialized")
        SMOKE_RESULTS["ui"] = "SKIP (no Megaton)"
    else:
        print("UI smoke: headless=False")
        try:
            from megaton.start import Megaton

            ui_input = os.environ.get("MEGATON_CREDS_JSON")
            if not ui_input:
                print("Skipped: MEGATON_CREDS_JSON not set")
                SMOKE_RESULTS["ui"] = "SKIP (no creds)"
            else:
                ui_app = Megaton(ui_input, headless=False)
                print("UI smoke OK: widgets initialized (no selection performed).")
                SMOKE_RESULTS["ui"] = "OK"
        except Exception as exc:
            print("UI smoke failed:", exc)
            SMOKE_RESULTS["ui"] = "FAIL (ui init)"
else:
    print("Skipped: RUN_UI_SMOKE is False")
    SMOKE_RESULTS["ui"] = "SKIP"

if RUN_OAUTH_SMOKE:
    if not GATE_OK:
        print("Skipped: required checks failed")
        SMOKE_RESULTS["oauth"] = "SKIP (gate)"
    else:
        print("OAuth smoke: up to auth prompt")
        try:
            from megaton.start import Megaton

            oauth_input = OAUTH_JSON or os.environ.get("SMOKE_OAUTH_JSON")
            if not oauth_input:
                print("Skipped: no OAuth JSON provided")
                SMOKE_RESULTS["oauth"] = "SKIP (no OAuth JSON)"
            else:
                Megaton(oauth_input, headless=False)
                print("OAuth smoke OK: auth prompt displayed (no completion).")
                SMOKE_RESULTS["oauth"] = "OK"
        except Exception as exc:
            print("OAuth smoke failed:", exc)
            SMOKE_RESULTS["oauth"] = "FAIL (oauth init)"
else:
    print("Skipped: RUN_OAUTH_SMOKE is False")
    SMOKE_RESULTS["oauth"] = "SKIP"


In [ ]:
# Cell 8: Summary
print("=== Smoke Test Summary ===")
for key in ["env", "init", "ga4", "sheets", "bq", "gsc", "drive", "ui", "oauth"]:
    if key in SMOKE_RESULTS:
        print(f"{key}: {SMOKE_RESULTS[key]}")

if GATE_OK:
    print("Status: PASS (required checks OK)")
else:
    print("Status: FAIL (required checks failed)")
